# 🔥 FHIR 101 - A Practical Guide

## Notebook Setup
If you haven't already setup your development environment please do so by following the instructions on the `kf-model-fhir` GitHub repository's [README](https://github.com/kids-first/kf-model-fhir#installation)

Next, run the cell below and then refresh this page in the browser.

In [7]:
%%capture
%%bash
pip install jupyter jupyterthemes jupyter_contrib_nbextensions
jupyter contrib nbextension install
jupyter nbextension enable collapsible_headings/main codefolding/main
jt -t solarizedl -cursw 3 -cellw 1000 -tf sourcesans

## Hello! What is this Guide?

If any of the following statements applies to you or you've had any of the questions below, then this guide is for you!
- 🤷‍♀️I've heard the term FHIR. But is it a server? A database? A data model? Rapid oxidation?
- 🙀I understand what FHIR is but have no idea how to implement it!
- 😕I tried reading through [HL7 FHIR](https://www.hl7.org/fhir/) but it is confusing and difficult to navigate
- 🤔How does FHIR modeling relate to relational databases?   

The primary purpose of this notebook is to quickly get you educated on what FHIR is and walk you through practical exercises that will teach you the FHIR basics:
1. Create and validate a FHIR data model for an entity
2. Add/remove simple and complex attributes to/from an entity in the model
3. Make entity attributes searchable in the FHIR server
4. Deploy the FHIR data model into a FHIR server
5. Load data into the FHIR server which conforms to the model
6. Search for data in the FHIR server

**If you don't care to know background and want to get right to development, head to the Tutorial section!**

## What is FHIR

Let's start with understanding exactly what FHIR is. FHIR stands for `Fast Healthcare Interoperability Resources` and was created by the organization `Health Level 7` or `HL7`.  

There is a large active international community that helps develop the specification. You can read more about it here: 

📘Learn more about [FHIR](http://hl7.org/fhir/index.html)

📘Learn more about [HL7](http://hl7.org)

### FHIR Is ...
A specification that includes all of the following:
- A loosely defined [base data model](https://www.hl7.org/fhir/resourcelist.html) describing things in healthcare (e.g. Patient, Specimen) and how they relate to     each other. The base data model also includes definitions and restrictions on server functionality
- How to extend or change the base data model to fit different healthcare use cases
- A database agnostic domain specific language (DSL) for developing files that make up the FHIR data model
- A [RESTful web API specification](https://www.hl7.org/fhir/exchange-module.html) to create, read, update, delete, and search FHIR data in a FHIR server
- A RESTful query language that describes how to construct search queries for a FHIR server
- Standards for how consumers of FHIR data should be authenticated and authorized access to health data.

### FHIR is NOT ...
- A database
- A database schema
- A server

It is important to understand that the FHIR specification is completely technology agnostic. Thus, it does not depend on programming languages or include things like relational database schemas. It is up to the implementers to decide how to implement the data model (i.e. relational database, nosql database, etc) and RESTful API. 

## Concepts

The FHIR spec is large and daunting, so we cannot cover all of the major concepts in FHIR. However, this section and the next should give you all you need to understand what you're doing in the tutorial. 

📘A good resource to learn [FHIR Basics](https://smilecdr.com/docs/tutorial_and_tour/fhir_basics.html#fhir-basics)

### FHIR Resource 

A term that describes a basic entity in the FHIR specification. A resource definition can be in JSON or XML format, but for the purposes of this guide, we will always use JSON.

📘Learn more about [FHIR Resources](https://www.hl7.org/fhir/resource.html)

**Examples of resources are:**

`Patient` - which represents a person in a healthcare system

`Specimen` - which represents a biosample drawn from a patient

`StructureDefinition` - this is different from the first two. It describes the model or definition for various
healthcare entities such as `Patient`, `Specimen`, etc.

**A `Patient` resource might look like this:**
```json
{
    "resourceType":"Patient",
    "id": "PT-001",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    },
    "gender": "male",
    "name": [
        {
            "use": "usual",
            "family":"Smith",
            "given": "Jack",
            "period": {
                "end": "2001-01-01"
            }
        }
    ]
}
``` 

### Resource Categories

There are different categories of FHIR resources such as 

- `Conformance Resource` 
- `Terminology Resource`
- `Individual Resource`
- `Diagnosic Resource`
- ...many more

However, the most important categories of resources for the purposes of this tutorial are `Conformance Resources` and `Terminology Resources` because these are the ones we will create in order to build a FHIR data model. 

### FHIR Data Types

FHIR resource models use a set of data types for holding their values. Data types can be primitive, meaning that they hold a simple value (typically a string, a number, etc.). Other data types are composite, meaning they are a structure containing several elements. Some examples are:

**Primitive Data Types**

- String - a string of text - `female`
- Decimal - an abitrary precision float - `3.14`

**Composite Data Types**

- HumanName - A person's complete name, broken into individual parts
```json
{
   "family": "Smith",
   "given": [ "John", "Frank" ],
   "suffix": [ "Jr" ]
}
```

📘Learn about all [FHIR Data Types](https://www.hl7.org/fhir/datatypes.html)

### FHIR Versions

Changes to the FHIR specification are tracked via different versions. A FHIR version has a name 
and a semantic version number. The biggest differences among versions are usually structural and content
changes in the FHIR base model definitions.

The most current and commonly used versions are:

- STU3 or version 3.0.2
- R4 or version 4.0.1

**This tutorial will always use version R4**

📘Learn more about [FHIR Versions](https://www.hl7.org/fhir/versions.html#versions)

## Model Development

What does it mean to develop a FHIR data model? Well, its not entirely different from developing any kind of data model. Developing a data model usually consists of the following steps: 

1. Define entities in the model and their attributes
2. Define relationships between entities
3. Define constraints on entity attributes (i.e. types, min/max, one of, etc)
4. Define constraints on entity relationships (i.e. Specimen must have one Patient, DiagnosticReport must have one      Patient, etc.)
5. Validate the model

More concretely, developing a data model in FHIR means:
1. Authoring JSON files containing `Conformance Resource` and `Terminology Resource` payloads
2. Validating `Conformance Resource` and `Terminology Resource` using one of the standalone HL7 validation tools or POSTing the resource payload to a FHIR server

⚡️**Key Take Away** - when we say "model" in FHIR, we are referring to `Conformance Resources` and `Terminology Resources`

### Conformance Resources

There are several different types of `Conformance Resources` but the ones that are used most in FHIR model development are: 

- `StructureDefinition` - this resource captures steps 1 through 4 above for healthcare entities like `Patient`, `Specimen`, `DiagnosticReport`, etc.

- `SearchParameter` - this resource is used to specify an attribute on an entity (Patient.gender) which can be used to search for instances of that entity on a FHIR server and what keyword/path (Patient.gender.code.value) should be used when searching.

We will take a deep dive into both of these in the tutorial.

📘Learn more about [Conformance Resources](https://www.hl7.org/fhir/conformance-module.html)

### Terminology Resources

Some entity attributes might need to be constrained to a particular value set which may come from a particular terminology system (or ontology) which defines attributes, their set of valid values, and an alphanumeric code that uniquely identifies each value.

For example, we may want to constrain the `bodySite` attribute on `Specimen` to only have values from the NCIT ontology's `Specimen Source Site` value set. We can do this by defining `Terminology Resource`s and referencing them in the `StructureDefinition` for `Specimen`.

Types of `Terminology Resource`s are:

- `CodeSystem` - describes an Ontology, Terminology, or Enumeration

- `ValueSet` - specifies a set of code and values drawn from one or more code systems

You will see a practical example of this later in the tutorial.

📘Learn about [Terminology Resources](https://www.hl7.org/fhir/terminologies.html)

### Model Documentation

#### Implementation Guide
FHIR data models are documented in a static website called an Implementation Guide. It has a standard format and looks much like the HL7 FHIR website. Implementation Guides describe and visually illustrate:

- The model's `StructureDefinition`s
- The differentials between the model's `StructureDefinition`s and the ones they extend
- Example Resources for each `StructureDefinition`
- The searchable attributes on each Resource

There is also a FHIR resource, `ImplementationGuide`, which is a type of `Conformance Resource` used to store configuration information for generating the static site. 

#### Create and Publish
Implementation Guides are generated using the [HL7 IG Publisher](https://confluence.hl7.org/display/FHIR/IG+Publisher+Documentation) Java CLI. The IG Publisher also fully validates the data model during its site generation process so it is often used as a model validation tool as well. The Kids First FHIR model uses it in this way.

📘Learn more about Kids First [IG generation and model validation](https://github.com/kids-first/kf-model-fhir/blob/master/README.md) using IG Publisher 

### Model Validation

There are 2 primary ways to validate:

1. Use the [HL7 IG Publisher](https://confluence.hl7.org/display/FHIR/IG+Publisher+Documentation) Java CLI
2. Use a FHIR server's `<resourceType>/$validate` endpoint

The most thorough and standard way to validate a FHIR data model is to use the IG Publisher. However, since the IG Publisher tightly couples IG site generation with validation it can be complicated to use. 

The second method is easier but not standard since each FHIR server may have slight differences in how is validated and how.

To keep things simple, we will use the second method of validation.

⚡️**Key Take Away** - the most thorough way to validate a FHIR data model is using the HL7 IG Publisher

## Server Development

### Pick an Existing Solution
You might have noticed that there are not many open source or free FHIR servers currently in development. This is likely due to a couple of factors:

- The FHIR specification is rapidly evolving and the tooling around it is still in early stages of development. 
- Server development is hard 

In order to develop a compliant FHIR server you must: 

- Implement the FHIR model validation logic. This is not easy since a FHIR model is authored     in the FHIR DSL. The server must know how to parse the DSL (a.k.a conformance resources) and   execute rules defined in them.
- Implement a parser and query constructor for the custom FHIR RESTful query language

### FHIR Server Solutions
For these reasons it is best to select one of the existing FHIR server solutions rather than develop your own. The most promising ones are:

<table style="bgcolor: none;">
    <tbody>
        <tr>
            <td style="border-right: 1px solid #cccccc; padding: 7px;">
                <a href="https://health-samurai.webflow.io/aidbox">
                    <img src="static/images/aidbox-logo.png" alt="Health Samurai Aidbox"/>
                </a>
            </td>
            <td style="padding: 7px;">
                <a href="https://smilecdr.com">
                    <img src="static/images/smilecdr-logo.png" alt="Smile CDR"/>
                </a>
            </td>
        </tr>
    </tbody>
</table>





## Tutorial

### Use Case
In this tutorial we will create a simple data model that is able to capture basic demographic information about Leukemia patients that have had a biosample drawn from them.  No PHI data should be captured. In fact there should be no protected health information [(PHI)](https://www.hipaajournal.com/considered-phi-hipaa/) attributes available in the model.

At a minimum we want to capture:

**Patient Attributes**
- Gender (required)
- Biological sex (required)

**Specimen Attributes**
- The patient from whom the sample was drawn
- Anatomical site or location on the body from which the sample was drawn
- Composition - the type of sample - blood, tissue, saliva, etc.

### Concepts
You will learn the following practical concepts in this tutorial:

- How to create a new model for a FHIR resource by extending it's base model
- How to add/remove attributes from the base model
- How to add make attributes required
- How to make an attribute searchable
- How to add a coded attribute - or one that is restricted to an enumeration defined by an ontology
- How to validate your model
- How to document your model

### Setup

#### Requirements
**FHIR Server**

- We will use [HAPI](http://hapi.fhir.org/), the reference FHIR server, for this tutorial. 

**Base URL**

- All HTTP requests will be sent to HAPI's base URL: http://hapi.fhir.org/baseR4

**Model Development Editor**

You will need something to author JSON formatted model files. Unfortunately there are almost no user friendly tools for this. Most model development is done in a code editor. This can be difficult as it requires the developer to unecessarily remember complex syntax of the FHIR DSL in order to create and edit model files.

- **Forge** - is a [visual editor](https://simplifier.net/forge) for FHIR model development.     It does make some things easier, but it can be buggy and was natively developed for windows.   It is possible to [run it on OSX](http://docs.simplifier.net/forge/forgeInstall.html#running-on-macos). You can use it, but       explanation of how to use it is out   of the scope of this tutorial.

#### Development Environment
Here we're going to import the code we need and complete the necessary setup which will be used later in the tutorial.

You don't need to do this if you would like to use another HTTP client like `Postman` to interact with the FHIR server.

In [23]:
# Be sure to execute this cell so that we can use the client later
from pprint import pprint
from kf_model_fhir.client import FhirApiClient
BASE_URL = 'http://hapi.fhir.org/baseR4'
FHIR_VERSION = '4.0.0'
client = FhirApiClient(base_url=BASE_URL, fhir_version=FHIR_VERSION)

### Create a Patient Model

The FHIR base model was intended to include many common healthcare entities
like patients, biospecimens, etc. which cover 80% of healthcare data capture
uses cases.

It is rare to come across a healthcare entity that isn't represented in
one of the existing FHIR resources. The remaining 20% of data capture use cases
that are not satisfied by the base model can easily be satisfied by extending
the base model.

So we can start by looking at the base model's `Patient` `StructureDefinition` to
see if anything needs to be modified.

The HL7 FHIR website has a nice visual representation of the `Patient`
`StructureDefinition`:

[![Patient StructureDefinition](static/images/patient-sd-visual.png)](https://www.hl7.org/fhir/patient.html#resource)

We can already see that the `Patient` `StructureDefinition` has one attribute we want:

- `gender`

And many attributes that we'll want to **remove** because they represent PHI fields:
- `name`
- `telecom`
- `address`
- `birthDate`
- `contactPoint`
- `photo`

#### Create Initial Patient StructureDefinition

To create a custom data model for `Patient`, we must create a `StructureDefinition` which 
will extend the FHIR R4 `StructureDefinition` for `Patient`:

In [7]:
# Be sure to execute this cell so that we can use `patient_sd` later
patient_sd = {
    "resourceType": "StructureDefinition",
    "url": "http://fhir.kids-first.io/StructureDefinition/Patient",
    "version": "0.1.0",
    "name": "kids_first_research_participant",
    "title": "Kids First Research Participant",
    "status": "draft",
    "publisher": "Kids First DRC",
    "description": "The individual human or other organism.",
    "fhirVersion": "4.0.0",
    "kind": "resource",
    "abstract": False,
    "type": "Patient",
    "baseDefinition": "http://hl7.org/fhir/StructureDefinition/Patient",
    "derivation": "constraint",
    "differential": {
        "element": [
            {
                "id": "Patient",
                "path": "Patient"
            },
            {
                "id": "Patient.identifier",
                "path": "Patient.identifier",
                "mustSupport": True
            },
            {
                "id": "Patient.name",
                "path": "Patient.name",
                "max": "0"
            },
            {
                "id": "Patient.telecom",
                "path": "Patient.telecom",
                "max": "0"
            },
            {
                "id": "Patient.address",
                "path": "Patient.address",
                "max": "0"
            },
            {
                "id": "Patient.photo",
                "path": "Patient.photo",
                "max": "0"
            },
            {
                "id": "Patient.contact",
                "path": "Patient.contact",
                "max": "0"
            },
            {
                "id": "Patient.gender",
                "path": "Patient.gender",
                "min": 1
            }
        ]
    }
}

#### Anatomy of StructureDefinition

We won't cover this in great detail but here are the important parts of the `StructureDefinition`:

- `resourceType` - every FHIR resource has to declare what type it is
- `baseDefinition` - this points to the `StructureDefinition` that the current one extends. It doesn't have to be one of the base FHIR definitions. It could be a `StructureDefinition`   from any FHIR data model. This is one of the features of FHIR that helps achieve interoperability.
- `url` - this is a required unique identifier for a `Conformance Resource`. It is a stable [canonical identifier](http://hl7.org/fhir/resource.html#canonical) for the resource and is never intended to change regardless of which FHIR server uses it.
- `fhirVersion` - the FHIR semantic version number is required. It will instruct  anything validating this resource to use R4 (4.0.0) validation logic.
- `differential` or `snapshot` - Every `StructureDefinition` must have either a `differential` - a section of content that specifies differences from the base definition, or a `snapshot` section which specifies the full definition. For model development, you almost always use the `differential` section and not the `snapshot`.

📘Learn more about [StructureDefinition](http://hl7.org/fhir/structuredefinition.html) anatomy

📘Learn more about [Resource identifiers](http://hl7.org/fhir/resource.html#identification)

📘Browse [FHIR data models](https://simplifier.net/search?searchtype=Projects) from other projects

### Remove Attributes

We're going to start with the easiest modification first: removing an attribute from the base model. In fact, in our initial `Patient` `StructureDefinition`, we've already removed a handful of attributes. 

To remove an attribute you simply set its maximum cardinality to 0 in the `StructureDefinition` `differential` section: 

```json
{
    "id": "Patient.name",
    "path": "Patient.name",
    "max": "0"
}
```

### Make an Attribute Required

This is just as simple as removing an attribute. To make an attribute required you simply set its minimum cardinality to 1 in the `StructureDefinition` `differential` section::

```json
{
    "id": "Patient.gender",
    "path": "Patient.gender",
    "min": 1
}
```

### Validate the Patient Model

Validating the Kids First Patient model means 2 things:

1. Validate the Kids First `Patient` `StructureDefinition` - structure, syntax, cardinality, etc.
2. Test both valid and invalid `Patient` Resources against the Kids First Patient `StructureDefinition`

📘Learn more about [Resource Validation](https://www.hl7.org/fhir/validation.html)

#### Validate StructureDefinition

In [28]:
endpoint = f'{BASE_URL}/StructureDefinition'

# Let's get into the habit of deleting existing instances before we create one
# The easiest way to do this is to search for StructureDefinitions by canonical URL and
# then delete what is returned (should only be 1 since canonical URL is a unique ID)
success = client.delete_all(endpoint, params={'url': patient_sd['url']})
assert success

# POST the Patient StructureDefinition /StructureDefinition endpoint
success, result = client.send_request('POST', endpoint, json=patient_sd)
assert success

#### Create a Patient Resource

In [58]:
# Let's start by creating a valid (has all required attributes) Patient resource
patient_rs = {
    "resourceType":"Patient",
    "id": "kidsfirst-test-patient",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    },
    "gender": "male"
}

#### Validate the Patient Resource

To test the `Patient` resource against the Kids First Patient model you must declare that this resource conforms to the `Patient` `StructureDefinition` like this:

```json
"meta": {
    "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
}
```

In [64]:
# Validate by POSTing to the /Patient/$validate endpoint
endpoint = f"{BASE_URL}/Patient/$validate"

# Since we know this is a valid Patient resource, it should succeed
success, result = client.send_request('POST', endpoint, json=patient_rs)
assert success

#### Validate an Invalid Patient Resource

In [66]:
# Let's create an invalid resource and make sure it fails 
# Invalid since its missing the REQUIRED gender attribute
patient_rs_invalid = {
    "resourceType":"Patient",
    "id": "kidsfirst-test-patient",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    }
}
success, result = client.send_request('POST', endpoint, json=patient_rs_invalid)
assert not success